## 计算问卷的竞争度

In [ ]:
import pandas as pd

# 读取 CSV 文件，替换为您的文件路径
file_path = '../竞争性感知调查结果.csv'  # 请替换为您的 CSV 文件路径
df = pd.read_csv(file_path, header=None)

# 初始化结果列表
final_results = []

# 每块数据的行数和小计列索引
rows_per_block = 10
subtotal_indices = [2, 6, 10, 14, 18]  # 小计列的索引（从0开始）

# 遍历每一块数据
for block_start in range(0, len(df), rows_per_block + 1):  # 加1是为了跳过空行
    # 提取当前块的数据
    block_data = df.iloc[block_start:block_start + rows_per_block, subtotal_indices]
    
    # 初始化内层结果数组
    inner_results = []
    
    # 提取每个小计列下的五个元素
    for col in block_data.columns:
        subtotal_col = block_data[col].dropna().astype(str).str.replace('小计', '', regex=False).str.strip()
        subtotal_col = pd.to_numeric(subtotal_col, errors='coerce')  # 转换为数值，无法转换的将变为 NaN
        subtotal_col = subtotal_col.dropna().astype(int).tolist()  # 去掉 NaN 并转换为整数
        
        # 确保提取到5个元素
        if len(subtotal_col) >= 5:
            inner_results.append(subtotal_col[:5])  # 取前5个元素
    
    # 将内层数组添加到最终结果列表
    final_results.append(inner_results)

# 计算加权和并输出结果
for idx, inner_array in enumerate(final_results):
    for arr in inner_array:
        weighted_sum = sum((i + 1) * value for i, value in enumerate(arr))  # 计算加权和
        result = weighted_sum / 27  # 除以27
        print(f"{result:.4f}")


## 将问卷竞争度写回原文件

In [ ]:
import pandas as pd

# 读取 CSV 文件，替换为您的文件路径
file_path = '../竞争性感知调查结果.csv'  # 请替换为您的 CSV 文件路径
df = pd.read_csv(file_path, header=None)

# 初始化结果列表
final_results = []

# 每块数据的行数和小计列索引
rows_per_block = 10
subtotal_indices = [2, 6, 10, 14, 18]  # 小计列的索引（从0开始）

# 遍历每一块数据
for block_start in range(0, len(df), rows_per_block + 1):  # 加1是为了跳过空行
    # 提取当前块的数据
    block_data = df.iloc[block_start:block_start + rows_per_block, subtotal_indices]
    
    # 初始化内层结果数组
    inner_results = []
    
    # 提取每个小计列下的五个元素
    for col in block_data.columns:
        subtotal_col = block_data[col].dropna().astype(str).str.replace('小计', '', regex=False).str.strip()
        subtotal_col = pd.to_numeric(subtotal_col, errors='coerce')  # 转换为数值，无法转换的将变为 NaN
        subtotal_col = subtotal_col.dropna().astype(int).tolist()  # 去掉 NaN 并转换为整数
        
        # 确保提取到5个元素
        if len(subtotal_col) >= 5:
            inner_results.append(subtotal_col[:5])  # 取前5个元素
    
    # 将内层数组添加到最终结果列表
    final_results.append(inner_results)

# 计算加权和并准备写入的结果
results_to_write = []
for inner_array in final_results:
    for arr in inner_array:
        weighted_sum = sum((i + 1) * value for i, value in enumerate(arr))  # 计算加权和
        result = weighted_sum / 27  # 除以27
        results_to_write.append(f"{result:.4f}")  # 收集结果

# 指定要写入的行
rows_to_write = [9, 20, 31, 42, 53, 64, 75, 86, 97, 108]  # 0-based index for rows 10, 21, ..., 109
columns_to_write = [2, 6, 10, 14, 18]  # 0-based index for columns 3, 7, 11, 15, 19

# 写入结果到指定的行和列
for i, result in enumerate(results_to_write):
    row_index = rows_to_write[i // len(columns_to_write)]
    col_index = columns_to_write[i % len(columns_to_write)]
    df.at[row_index, col_index] = result

# 将结果写回原始 CSV 文件
df.to_csv(file_path, index=False, header=False)

print("结果已写回原始文件。")

## 每个竞争关键词竞争度对比

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 通过Compkey算法计算出的竞争度数据
compkey_data = {
    "图片": [2.7037, 2.4815, 2.7407, 4.3333, 3.3704],
    "手机": [2.7778, 3.2593, 4.0000, 3.3333, 3.7407],
    "小说": [3.1852, 4.0000, 2.6667, 4.2593, 2.8148],
    "视频": [3.8148, 3.3704, 3.8889, 3.6296, 3.3333],
    "下载": [2.5185, 4.0741, 3.9259, 3.6296, 2.8519],
    "大全": [3.7778, 2.8519, 3.6296, 2.4074, 2.1852],
    "qq": [3.4444, 2.7778, 3.8889, 2.5556, 4.0370],
    "电影": [3.3704, 3.0000, 3.5556, 3.9630, 3.2963],
    "中国": [3.0000, 2.6667, 3.7778, 3.6667, 3.1111],
    "世界": [3.2963, 2.8519, 3.5185, 3.1481, 2.0370]
}

# 通过问卷调查得出的用户感知的竞争度
survey_data = {
    "图片": {
        "做法": 0.01326591817790275,
        "句子": 0.0037486569289477997,
        "适合": 0.0032503498134106162,
        "视频": 0.00296876498462408,
        "价格表": 0.0019494069710063071
    },
    "手机": {
        "完整版": 0.011087185977932536,
        "头像": 0.006281341113089849,
        "荣耀": 0.005336954403822466,
        "6s": 0.0029454871298048566,
        "下载": 0.0025916884104219854
    },
    "小说": {
        "微微一笑": 0.019087612543686155,
        "全文": 0.012711982049501715,
        "火线": 0.009679589345428733,
        "女主角": 0.009080096719398294,
        "2016": 0.0050981174355383164
    },
    "视频": {
        "在线": 0.023647268862071004,
        "王者荣耀": 0.013764267081884163,
        "观看": 0.009877018137089709,
        "图片": 0.005036802112810619,
        "设计": 0.0029602638434619074
    },
    "下载": {
        "重生": 0.016387937933296588,
        "百度云": 0.013025669923698477,
        "阅读": 0.009763689917113091,
        "电视剧": 0.006951511960764893,
        "官方网站": 0.006759298266478977
    },
    "大全": {
        "小说": 0.011909053541637602,
        "家常": 0.008785348871345532,
        "发型": 0.004709256783675848,
        "倾城": 0.0031993373068572436,
        "好听": 0.0026071001346940573
    },
    "qq": {
        "小说": 0.020896654614101937,
        "侠盗": 0.014471638672742527,
        "手机版": 0.012159970472186872,
        "重生": 0.008515575181704501,
        "163": 0.005905945644497524
    },
    "电影": {
        "在线": 0.02794850642744299,
        "倾城": 0.017327600214331995,
        "微微一笑": 0.015285158128568533,
        "观看": 0.011408993921732485,
        "图片": 0.0076959165144257664
    },
    "中国": {
        "最新": 0.004990909059934353,
        "百度": 0.002518445620999911,
        "大学排名": 0.002183475052086682,
        "学院": 0.0007306053206919491,
        "教学": 0.00046128424994160307
    },
    "世界": {
        "魔兽争霸": 0.006913277562477582,
        "wifi": 0.006196371214168433,
        "异界": 0.0052397930585494,
        "手机版": 0.003983771192591188,
        "告白": 0.0031352980932548547
    }
}

# 归一化函数
def normalize(data):
    data = np.array(data)
    return (data - np.min(data)) / (np.max(data) - np.min(data))

# 归一化处理
normalized_compkey_data = []
normalized_survey_data = []

for seed_keyword in compkey_data.keys():
    normalized_compkey_data.extend(normalize(compkey_data[seed_keyword]))
    normalized_survey_data.extend(normalize(list(survey_data[seed_keyword].values())))

# 绘制对比折线图
plt.figure(figsize=(15, 8))

# 横轴的竞争关键词数量
num_keywords = len(normalized_compkey_data)

# 绘制算法和问卷的竞争度
plt.plot(np.arange(num_keywords), normalized_compkey_data, marker='o', label='Compkey Algorithm', color='blue')
plt.plot(np.arange(num_keywords), normalized_survey_data, marker='x', linestyle='--', label='Survey Data', color='orange')

# 设置图表标题和标签
plt.title("竞争度对比：Compkey算法 vs 问卷调查")
plt.xlabel("竞争关键词")
plt.ylabel("归一化竞争度")
plt.xticks(ticks=np.arange(num_keywords), labels=[f'Keyword {i+1}' for i in range(num_keywords)], rotation=45)
plt.legend()
plt.grid()

# 显示图形
plt.tight_layout()
plt.show()


## 竞争度平均值对比

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['font.family'] ='sans-serif'
plt.rcParams['axes.unicode_minus'] = False

# 通过算法得出的用户感知的竞争度
compkey_data = {
    "图片": {
        "做法": 0.01326591817790275,
        "句子": 0.0037486569289477997,
        "适合": 0.0032503498134106162,
        "视频": 0.00296876498462408,
        "价格表": 0.0019494069710063071
    },
    "手机": {
        "完整版": 0.011087185977932536,
        "头像": 0.006281341113089849,
        "荣耀": 0.005336954403822466,
        "6s": 0.0029454871298048566,
        "下载": 0.0025916884104219854
    },
    "小说": {
        "微微一笑": 0.019087612543686155,
        "全文": 0.012711982049501715,
        "火线": 0.009679589345428733,
        "女主角": 0.009080096719398294,
        "2016": 0.0050981174355383164
    },
    "视频": {
        "在线": 0.023647268862071004,
        "王者荣耀": 0.013764267081884163,
        "观看": 0.009877018137089709,
        "图片": 0.005036802112810619,
        "设计": 0.0029602638434619074
    },
    "下载": {
        "重生": 0.016387937933296588,
        "百度云": 0.013025669923698477,
        "阅读": 0.009763689917113091,
        "电视剧": 0.006951511960764893,
        "官方网站": 0.006759298266478977
    },
    "大全": {
        "小说": 0.011909053541637602,
        "家常": 0.008785348871345532,
        "发型": 0.004709256783675848,
        "倾城": 0.0031993373068572436,
        "好听": 0.0026071001346940573
    },
    "qq": {
        "小说": 0.020896654614101937,
        "侠盗": 0.014471638672742527,
        "手机版": 0.012159970472186872,
        "重生": 0.008515575181704501,
        "163": 0.005905945644497524
    },
    "电影": {
        "在线": 0.02794850642744299,
        "倾城": 0.017327600214331995,
        "微微一笑": 0.015285158128568533,
        "观看": 0.011408993921732485,
        "图片": 0.0076959165144257664
    },
    "中国": {
        "最新": 0.004990909059934353,
        "百度": 0.002518445620999911,
        "大学排名": 0.002183475052086682,
        "学院": 0.0007306053206919491,
        "教学": 0.00046128424994160307
    },
    "世界": {
        "魔兽争霸": 0.006913277562477582,
        "wifi": 0.006196371214168433,
        "异界": 0.0052397930585494,
        "手机版": 0.003983771192591188,
        "告白": 0.0031352980932548547
    }
}

# 通过问卷调查得出的竞争度数据
survey_data = {
    "图片": [2.7037, 2.4815, 2.7407, 4.3333, 3.3704],
    "手机": [2.7778, 3.2593, 4.0000, 3.3333, 3.7407],
    "小说": [3.1852, 4.0000, 2.6667, 4.2593, 2.8148],
    "视频": [3.8148, 3.3704, 3.8889, 3.6296, 3.3333],
    "下载": [2.5185, 4.0741, 3.9259, 3.6296, 2.8519],
    "大全": [3.7778, 2.8519, 3.6296, 2.4074, 2.1852],
    "qq": [3.4444, 2.7778, 3.8889, 2.5556, 4.0370],
    "电影": [3.3704, 3.0000, 3.5556, 3.9630, 3.2963],
    "中国": [3.0000, 2.6667, 3.7778, 3.6667, 3.1111],
    "世界": [3.2963, 2.8519, 3.5185, 3.1481, 2.0370]
}

# 计算平均值函数
def calculate_average(data):
    return {key: np.mean(value) for key, value in data.items()}

# 归一化函数
def normalize(data):
    min_val = np.min(data)
    max_val = np.max(data)
    return (data - min_val) / (max_val - min_val)

# 计算平均竞争度
average_survey = calculate_average({key: list(value.values()) for key, value in compkey_data.items()})
average_compkey = calculate_average(survey_data)

# 提取种子关键词和对应的平均值
keywords = list(average_survey.keys())
survey_averages = np.array([average_survey[key] for key in keywords])
compkey_averages = np.array([average_compkey[key] for key in keywords])

# 归一化处理
survey_averages_normalized = normalize(survey_averages)
compkey_averages_normalized = normalize(compkey_averages)

# 绘制对比折线图
plt.figure(figsize=(12, 6))

# 绘制问卷和算法的竞争度
plt.plot(keywords, survey_averages_normalized, marker='o', label='问卷调查数据（归一化）', color='blue')
plt.plot(keywords, compkey_averages_normalized, marker='x', linestyle='--', label='算法数据（归一化）', color='orange')

# 设置图表标题和标签
plt.title("竞争度对比：算法数据 vs 问卷调查数据（归一化处理）")
plt.xlabel("种子关键词")
plt.ylabel("归一化平均竞争度")
plt.xticks(rotation=45)
plt.grid()

# 设置 y 轴范围
plt.ylim(0, 1)  # 根据需要调整 y 轴的范围

# 添加图例
plt.legend()

# 显示图形
plt.tight_layout()
plt.show()


In [ ]:
survey_averages_normalized

In [ ]:
compkey_averages_normalized